## Задача. Сделать mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм

### 1 шаг. Подготовка окружения

In [13]:
!pip install opendatasets

### 2 шаг. Загрузка библиотек

In [14]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

from functools import reduce
import json

### 3 шаг. Скачивание датасета

In [15]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: anastasiakov
Your Kaggle Key: ········


100%|██████████████████████████████████████| 15.1M/15.1M [00:02<00:00, 7.30MB/s]


### 4 шаг. Реализация через цикл 

(предпогаем, что мы не знаем сколько у нас фильмов):

In [20]:
n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774867


### 5 шаг. На основе этого кода соберите mapper и reducer:

In [80]:
# достаем рейтинг из файла
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        # записываем json-файл в переменную file
        with open(path, "r") as file:
            # .load(file) - записываем данные из json-файла в переменную info
            info = json.load(file)
            score = (info["movieIMDbRating"])
        # score, - без запятой не возвращает, выдает ошибку  
        return (score,)

    
def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        scores = []
        if len(score_data1) == 1:
            n, mean, s = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, s = score_data1
        scores.append(score_data2[0])        
        for score in scores:
            # n - кол-во оценок
            n += 1
            # delta - разница между оценкой и средней оценкой
            delta = score - mean
            # средняя оценка
            mean += delta / n
            # s - сумма всех разностей между оценками и средней оценкой
            s += (score - mean) * delta
        return n, mean, s

In [81]:
%%time
n, mean, s = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))

print("среднее =", mean)   
print("дисперсия =", (s / n))    
print("стандартное отклонение =", (s / n) ** (1/2))   

среднее = 8.030000000000001
дисперсия = 1.106100000000001
стандартное отклонение = 1.0517128885774867
CPU times: user 3.16 ms, sys: 4.78 ms, total: 7.94 ms
Wall time: 8.82 ms
